<a href="https://colab.research.google.com/github/Gynic/iu5-MMO-lab/blob/main/MMO_PK2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ИУ5И-23М Фу Чэньтянь

загрузка набора данных

In [1]:
from sklearn.datasets import fetch_20newsgroups

# В следующем примере используется sklearn.datasets.fetch_20newsgroups
categories = ['talk.politics.misc', 'rec.sport.hockey', 'sci.med']
data = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))

X_raw, y = data.data, data.target


Функция извлечения признаков и оценки моделирования

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

def evaluate_model(vectorizer, classifier, X, y):
    pipe = Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', classifier)
    ])
    scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
    print(f"{classifier.__class__.__name__} + {vectorizer.__class__.__name__} Accuracy: {scores.mean():.4f}")
    pipe.fit(X, y)
    y_pred = pipe.predict(X)
    print(classification_report(y, y_pred, target_names=data.target_names))


оценка CoutVectorizer и TfidfVectorizer

In [3]:
print("🔹 CountVectorizer ：")
evaluate_model(CountVectorizer(), LogisticRegression(max_iter=1000), X_raw, y)
evaluate_model(CountVectorizer(), LinearSVC(), X_raw, y)

print("🔸 TfidfVectorizer ：")
evaluate_model(TfidfVectorizer(), LogisticRegression(max_iter=1000), X_raw, y)
evaluate_model(TfidfVectorizer(), LinearSVC(), X_raw, y)


🔹 CountVectorizer ：
LogisticRegression + CountVectorizer Accuracy: 0.8595
                    precision    recall  f1-score   support

  rec.sport.hockey       0.95      1.00      0.98       600
           sci.med       1.00      0.97      0.99       594
talk.politics.misc       1.00      0.97      0.99       465

          accuracy                           0.98      1659
         macro avg       0.98      0.98      0.98      1659
      weighted avg       0.98      0.98      0.98      1659



/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC + CountVectorizer Accuracy: 0.8336
                    precision    recall  f1-score   support

  rec.sport.hockey       0.95      1.00      0.98       600
           sci.med       1.00      0.97      0.99       594
talk.politics.misc       1.00      0.97      0.99       465

          accuracy                           0.98      1659
         macro avg       0.98      0.98      0.98      1659
      weighted avg       0.98      0.98      0.98      1659

🔸 TfidfVectorizer ：
LogisticRegression + TfidfVectorizer Accuracy: 0.9023
                    precision    recall  f1-score   support

  rec.sport.hockey       0.95      1.00      0.98       600
           sci.med       0.99      0.97      0.98       594
talk.politics.misc       1.00      0.97      0.98       465

          accuracy                           0.98      1659
         macro avg       0.98      0.98      0.98      1659
      weighted avg       0.98      0.98      0.98      1659

LinearSVC + TfidfVectorizer Accurac

В этом эксперименте мы выбрали три категории (rec.sport.hockey, sci.med, talk.politics.misc) из набора данных 20 Newsgroups и выполнили классификацию текста на них. Цель состоит в том, чтобы сравнить эффекты классификации различных методов векторизации текста (CountVectorizer и TfidfVectorizer) в сочетании с различными классификаторами (LogisticRegression и LinearSVC), чтобы найти комбинацию с наилучшей производительностью.

Общая точность:

Все четыре модели достигли точности 0,98 на тестовом наборе, что указывает на то, что проблема мультиклассификации относительно линейно разделима.

Однако средняя точность перекрестной проверки (точность CV) может лучше отражать обобщающую способность модели.

Сравнение методов векторизации:

Точность перекрестной проверки TfidfVectorizer в обоих классификаторах (0,9023 и 0,9186) выше, чем у CountVectorizer (0,8595 и 0,8336).

Это показывает, что механизм взвешивания частоты слов TF-IDF больше подходит для текущей задачи классификации текста и может лучше различать слова в разных контекстах.

Сравнение классификаторов:

В CountVectorizer LogisticRegression значительно лучше, чем LinearSVC, который также имеет предупреждение о сходимости и нуждается в корректировке количества итераций.

При использовании TfidfVectorizer LinearSVC превосходит LogisticRegression и имеет самую высокую точность перекрестной проверки (0,9186) среди всех комбинаций.


Лучшая комбинация моделей: TfidfVectorizer + LinearSVC, которая стабильно и хорошо работает как при перекрестной проверке, так и при окончательном тестировании.

Результаты сравнения показывают, что TF-IDF имеет больше преимуществ при классификации текста, особенно в сочетании с линейной моделью опорных векторов (SVC).